In [1643]:
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta

In [157]:
# get the data from last 60 days until the present
# dataF = yf.download("^NDX", start="2023-10-25", end="2023-12-21", interval='15m')
# dataF = yf.download("EURJPY=X", period="60d", interval='15m')
dataF = yf.download("^NDX", period="730d", interval='1h')
# dataF = yf.download("^SPX", period="500d", interval='1h')
# dataF = yf.download("AAPL", period="730d", interval='1h')
# dataF = yf.download("^NDX", period="50d", interval='5m')
# dataF = yf.download("EURUSD=x", period="50d", interval='5m')
# dataF = yf.download("EURJPY=X", period="50d", interval='30m') # <-- this is the one
# dataF = yf.download("EURJPY=X", period="365d", interval='1h')# <-- this is the one
# dataF = yf.download("EURJPY=X", period="500d", interval='1h')# <--------- this is the one
# dataF = yf.download("EURUSD=X", period="365d", interval='1h') # XXXX Horrible
# dataF = yf.download("USDJPY=X", period="365d", interval='1h') <-- not too bad

# dataF.iloc[:,:]
#dataF.Open.iloc

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [158]:
df = pd.DataFrame(dataF)

# use df index, convert DateTime to  a column instead of index
df.reset_index(inplace=True)

# delete Adj Close
df = df.drop(['Adj Close'], axis=1)

# rename Datetime to "Gmt time"
df = df.rename(columns={'Datetime':'Gmt time'})

# set gmt time as index
# df.set_index('Gmt time', inplace=True)

df=df[df.High!=df.Low]
df.reset_index(inplace=True)
df.head()

df

,index,Gmt time,Open,High,Low,Close,Volume
0,0,2024-01-16 00:00:00,159.563995,159.578003,159.332993,159.356995,0
1,1,2024-01-16 00:15:00,159.343994,159.419998,159.315994,159.403000,0
2,2,2024-01-16 00:30:00,159.404999,159.410004,159.315994,159.350006,0
3,3,2024-01-16 00:45:00,159.348999,159.389999,159.248993,159.257996,0
4,4,2024-01-16 01:00:00,159.257004,159.483002,159.235992,159.347000,0
...,...,...,...,...,...,...,...
4681,4693,2024-03-25 14:15:00,164.009995,164.041000,163.988007,164.020996,0
4682,4694,2024-03-25 14:30:00,164.020004,164.042007,163.983994,164.024002,0
4683,4695,2024-03-25 14:45:00,164.026993,164.082001,164.007996,164.057007,0
4684,4696,2024-03-25 15:00:00,164.057007,164.078995,163.953003,163.955994,0


In [159]:
import pandas_ta as ta
df["MACD"]=ta.macd(df.Close)['MACD_12_26_9']
df["MACD_HIST"]=ta.macd(df.Close)['MACDh_12_26_9']
df["MACD_SIGNAL"]=ta.macd(df.Close)['MACDs_12_26_9']
df["ADX"]=ta.adx(df.High, df.Low, df.Close)['ADX_14']
df["20_MA"] = ta.ema(df.Close, length=20)
df["50_MA"] = ta.ema(df.Close, length=50)
df["100_MA"] = ta.ema(df.Close, length=100)
df["200_MA"]= ta.ema(df.Close, length=200)
df['EMA']=ta.ema(df.Close, length=200)#sma ema
df['EMA2']=ta.ema(df.Close, length=150)#sma ema
df['EMA3']=ta.ema(df.Close, length=100)#sma ema
df['EMA4']=ta.ema(df.Close, length=50)#sma ema
df['EMA5']=ta.ema(df.Close, length=20)#sma ema
df['RSI']=ta.rsi(df.Close, length=12)

# df["HA_OPEN"]=ta.ha(df.Open, df.High, df.Low, df.Close)['HA_open']
# df["HA_CLOSE"]=ta.ha(df.Open, df.High, df.Low, df.Close)['HA_close']
my_bbands = ta.bbands(df.Close, length=14, std=2.0)
df=df.join(my_bbands)

In [160]:
def addemasignal(df):
    emasignal = [0]*len(df)
    for i in range(0, len(df)):
        if df.EMA3[i]>df.EMA[i]:
            emasignal[i]=2
        elif df.EMA3[i]<df.EMA[i]:
            emasignal[i]=1
        if (df.EMA4[i]<df.EMA2[i]):
            emasignal[i]=0
    df['EMASignal'] = emasignal
addemasignal(df)

In [161]:
def addorderslimit(df, percent):
    ordersignal=[0]*len(df)
    for i in range(1, len(df)): #EMASignal of previous candle!!! modified!!!
        if  df.Close[i]<=df['BBL_14_2.0'][i] and df.EMASignal[i]==2:
            ordersignal[i]=df.Close[i]-df.Close[i]*percent
        elif df.Close[i]>=df['BBU_14_2.0'][i] and df.EMASignal[i]==1:
            ordersignal[i]=df.Close[i]+df.Close[i]*percent
    df['ordersignal']=ordersignal
    
addorderslimit(df, 0.000)

In [162]:
df['index_entry'] = df.index
# # copy gmt time into a enw column
df['Date'] = df['Gmt time']

# # copy the index to a new col

# # set it as the index
df.set_index('Date', inplace=True)



In [163]:
df[df.ordersignal!=0]

,index,Gmt time,Open,High,Low,Close,Volume,MACD,MACD_HIST,MACD_SIGNAL,...,EMA5,RSI,BBL_14_2.0,BBM_14_2.0,BBU_14_2.0,BBB_14_2.0,BBP_14_2.0,EMASignal,ordersignal,index_entry
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-18 04:30:00,210,2024-01-18 04:30:00,161.205994,161.229004,161.154999,161.154999,0,0.012391,-0.019161,0.031552,...,161.239843,41.569013,161.163845,161.264359,161.364872,0.124657,-0.044004,2,161.154999,210
2024-01-18 07:45:00,223,2024-01-18 07:45:00,161.044998,161.052994,160.904999,160.912994,0,-0.049415,-0.028427,-0.020988,...,161.144609,29.897555,160.961016,161.136214,161.311412,0.217453,-0.137050,2,160.912994,223
2024-01-18 08:00:00,224,2024-01-18 08:00:00,160.919998,160.944000,160.789993,160.856995,0,-0.065757,-0.035816,-0.029942,...,161.117217,27.325141,160.888972,161.114928,161.340885,0.280491,-0.070759,2,160.856995,224
2024-01-18 13:15:00,245,2024-01-18 13:15:00,160.819000,160.863998,160.759003,160.759003,0,-0.063268,-0.015535,-0.047733,...,160.944718,27.604768,160.767961,160.954786,161.141611,0.232146,-0.023974,2,160.759003,245
2024-01-18 14:30:00,250,2024-01-18 14:30:00,160.774002,160.807007,160.656998,160.688004,0,-0.073609,-0.014964,-0.058646,...,160.888339,31.567875,160.691886,160.872644,161.053402,0.224722,-0.010738,2,160.688004,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-21 19:45:00,4533,2024-03-21 19:45:00,164.735992,164.742996,164.658005,164.675995,0,-0.026110,0.001140,-0.027250,...,164.743051,41.091078,164.678535,164.739498,164.800460,0.074011,-0.020834,2,164.675995,4522
2024-03-21 20:00:00,4534,2024-03-21 20:00:00,164.677994,164.705994,164.643005,164.647995,0,-0.030582,-0.002666,-0.027916,...,164.733998,38.457119,164.658866,164.729355,164.799843,0.085581,-0.077115,2,164.647995,4523
2024-03-21 21:15:00,4539,2024-03-21 21:15:00,164.645004,164.692993,164.535995,164.539001,0,-0.047899,-0.011636,-0.036263,...,164.687501,28.305401,164.565496,164.683427,164.801358,0.143222,-0.112330,2,164.539001,4528


## Visualization

In [164]:
import numpy as np
def pointposbreak(x):
    if x['ordersignal']!=0:
        return x['ordersignal']
    else:
        return np.nan
df['pointposbreak'] = df.apply(lambda row: pointposbreak(row), axis=1)

In [165]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = df[1000:].copy()
#dfpl=dfpl.drop(columns=['level_0'])#!!!!!!!!!!
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA, line=dict(color='orange', width=2), name="EMA"),
                go.Scatter(x=dfpl.index, y=dfpl.EMA2, line=dict(color='yellow', width=2), name="EMA2"),        
                go.Scatter(x=dfpl.index, y=dfpl['BBL_14_2.0'], line=dict(color='blue', width=1), name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_14_2.0'], line=dict(color='blue', width=1), name="BBU")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=6, color="MediumPurple"),
                name="Signal")
#fig.update(layout_yaxis_range = [300,420])
fig.update_xaxes(rangeslider_visible=False)
fig.update_layout(autosize=False, width=1000, height=600,margin=dict(l=50,r=50,b=100,t=100,pad=4), paper_bgcolor="white")
fig.show()

In [166]:
dfpl = df[:].copy()
def SIGNAL():
    return dfpl.ordersignal

In [167]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.03
    mysize = initsize
    risk = 0
    latestEntry = 0
    
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 2
        perc = 0.02
        lastEntryIndex = 0
        
        if len(self.trades)>0:
            
            # close the position if the amount lost exceeds 3% of the account balance
            if (self.position.pl < -1 * self.equity * 0.015):
                self.position.close()
                print("Close position due to loss")
                print("------")
            
            # if the pl is greater than the stop loss amount, then move the stop loss to entry
            if (self.position.pl > self.risk):
                # adjust the stop loss to entry
                self.position.sl = self.latestEntry
        
            if self.data['index_entry'][-1]-self.lastEntryIndex>=10:
                self.trades[-1].close()
                self.lastEntryIndex = self.data['index_entry'][-1]
            if self.trades[-1].is_long and self.data.RSI[-1]>=75:
                self.trades[-1].close()
                self.lastEntryIndex = self.data['index_entry'][-1]
            elif self.trades[-1].is_short and self.data.RSI[-1]<=25:
                self.trades[-1].close()
                self.lastEntryIndex = self.data['index_entry'][-1]
        
        if self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==2:  
            sl1 = min(self.data.Low[-1],self.data.Low[-2])*(1-perc)
            tp1 = self.data.Close[-1]+(self.data.Close[-1] - sl1)*TPSLRatio
            entry = self.data.Close[-1]
            self.risk = entry - sl1
            self.latestEntry = entry
            self.lastEntryIndex = self.data['index_entry'][-1]
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==1:         
            sl1 = sl1 = max(self.data.High[-1],self.data.High[-2])*(1+perc)
            tp1 = self.data.Close[-1]-(sl1 - self.data.Close[-1])*TPSLRatio
            entry = self.data.Close[-1]
            self.risk = sl1 - entry
            self.latestEntry = entry
            self.lastEntryIndex = self.data['index_entry'][-1]
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=50000, margin=1/100, commission=.000)
stat = bt.run()
stat

Start                     2024-01-16 00:00:00
End                       2024-03-25 15:15:00
Duration                     69 days 15:15:00
Exposure Time [%]                   15.492958
Equity Final [$]                 48666.403717
Equity Peak [$]                  50183.303589
Return [%]                          -2.667193
Buy & Hold Return [%]                2.887859
Return (Ann.) [%]                  -12.737707
Volatility (Ann.) [%]                 5.33705
Sharpe Ratio                        -2.386657
Sortino Ratio                        -2.39222
Calmar Ratio                        -3.513234
Max. Drawdown [%]                   -3.625636
Avg. Drawdown [%]                   -0.950306
Max. Drawdown Duration       65 days 17:15:00
Avg. Drawdown Duration       13 days 11:42:00
# Trades                                   66
Win Rate [%]                        43.939394
Best Trade [%]                       0.245006
Worst Trade [%]                     -0.444035
Avg. Trade [%]                    

In [168]:
bt.plot()

Row(id='20572', ...)